# CompKey 竞争性关键词推荐算法设计与实现

In [43]:
import time

seedwords = ['图片', '手机', '小说', '视频', '下载', '大全', 'qq', '电影', '中国', '世界']
# , '重生', '百度', '官网','txt', '英语', '电视剧', '游戏', '查询', '做法', '倾城']

## 确定竞争性关键字集合

In [44]:
# 读取种子关键词和中介关键词的文件并转换成字典
def read_file_to_dict(file_path: str):
    data_dict = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        current_key = None
        for line in file:
            line = line.strip()
            if line.endswith(':'):
                current_key = line[:-1]
                data_dict[current_key] = []
            else:
                parts = line.split()
                if len(parts) == 3:
                    keyword, freq, weight = parts
                    data_dict[current_key].append({
                        'keyword': keyword,
                        'freq': int(freq),
                        'weight': float(weight)
                    })
    return data_dict


file_path = '../data/temp/seed_mid.train'
midkeys_dict = read_file_to_dict(file_path)
# 只要前十条种子关键词的数据
midkeys_dict = {key: midkeys_dict[key] for key in seedwords}

# 如果中介关键词包含种子关键词子串，则删除
for seedword in seedwords:
    midkeys_dict[seedword] = [midkey for midkey in midkeys_dict[seedword] if seedword not in midkey['keyword']]

# 只要前10个中介关键词
midkeys_dict = {key: midkeys_dict[key][:10] for key in midkeys_dict}

midkeys_dict

{'图片': [{'keyword': '大全', 'freq': 32220, 'weight': 0.10696500896354824},
  {'keyword': '发型', 'freq': 5682, 'weight': 0.018863289290219774},
  {'keyword': '报价', 'freq': 4264, 'weight': 0.014155766549365912},
  {'keyword': '唯美', 'freq': 4170, 'weight': 0.013843702277405218},
  {'keyword': '价格', 'freq': 4013, 'weight': 0.013322488546577253},
  {'keyword': '搞笑', 'freq': 3684, 'weight': 0.012230263594714826},
  {'keyword': '2016', 'freq': 3662, 'weight': 0.012157227275745303},
  {'keyword': '手机', 'freq': 3390, 'weight': 0.011254232786667552},
  {'keyword': 'qq', 'freq': 3246, 'weight': 0.010776176880685214},
  {'keyword': '动漫', 'freq': 3145, 'weight': 0.01044087377996149}],
 '手机': [{'keyword': '版', 'freq': 23242, 'weight': 0.10907230815437753},
  {'keyword': '下载', 'freq': 11998, 'weight': 0.05630537618261},
  {'keyword': 'qq', 'freq': 11581, 'weight': 0.05434843820393453},
  {'keyword': '苹果', 'freq': 10207, 'weight': 0.047900397957651296},
  {'keyword': '软件', 'freq': 7257, 'weight': 0.03405

In [45]:
# 筛选出不含种子关键词，但含有其中介关键词的搜索数据，存储在相应的文件中
def comkey_words_file(seedword: str):
    query_data = open('../data/temp/cleaned.train', 'r', encoding='utf-8')
    filename = ('../data/temp/compkey_') + seedword
    result_data = open(filename, 'w', encoding='utf-8')
    for line in query_data:
        if seedword not in line:
            for midkeyword in midkeys_dict[seedword]:
                if midkeyword['keyword'] in line:
                    result_data.write(line)
                    break
    query_data.close()
    result_data.close()


for seedword in seedwords:
    print(seedword)
    comkey_words_file(seedword)

图片
手机
小说
视频
下载
大全
qq
电影
中国
世界


In [46]:

# 1. 加载停用词
stopwords_file = '../data/stop_words/merge_stopwords.txt'  # 停用词文件路径
stopwords = set()
with open(stopwords_file, 'r', encoding='utf-8') as file:
    for line in file:
        stopwords.add(line.strip())
stopwords

{'',
 '向',
 '般的',
 'described',
 '遭到',
 '赶',
 'thereafter',
 'several',
 '一方面',
 "didn't",
 '别',
 '恰恰相反',
 '下',
 '尔尔',
 '大',
 '容易',
 'think',
 '进步',
 '突然',
 '产生',
 'ask',
 '啐',
 '但',
 '普通',
 "won't",
 '针对',
 'much',
 'further',
 '竟而',
 'on',
 '每当',
 'seeming',
 'about',
 "can't",
 '是以',
 '无',
 'of',
 '丰富',
 '去',
 'being',
 'here',
 'latter',
 'be',
 'somewhere',
 'best',
 '造成',
 '如此',
 '乃',
 '什麽',
 'zz',
 'know',
 '呃',
 '呗',
 'following',
 '哪年',
 '一定',
 'who',
 'relatively',
 'always',
 '自己',
 'via',
 '即令',
 '哪个',
 '了解',
 'becomes',
 '否则',
 'there',
 'regarding',
 'whoever',
 '即如',
 '不外乎',
 '向使',
 '难道说',
 '二来',
 'despite',
 "ain't",
 '可见',
 '哪儿',
 'inasmuch',
 '立即',
 '引起',
 'anywhere',
 'happens',
 '不久',
 '为什麽',
 '它们的',
 '果然',
 'his',
 '好',
 'some',
 '如其',
 '受到',
 '固然',
 'until',
 '今天',
 '哪里',
 '维持',
 'keeps',
 '几时',
 '咧',
 'from',
 'inner',
 '不变',
 '专门',
 'new',
 'welcome',
 'rd',
 '向着',
 '归齐',
 '正如',
 '她们',
 '明显',
 'becoming',
 '又及',
 'far',
 'around',
 '正巧',
 '主张',
 '普遍',
 '某',
 '：'

In [47]:
from collections import Counter


def stop_words_filter(word_list):
    word_cleaned = []
    for word in word_list:
        if word not in stopwords and word.strip() != '':
            word_cleaned.append(word)
    return word_cleaned
    # word_cleaned = []
    # for word in word_list:
    #     if word not in stopwords:
    #         word_cleaned.append(word)
    # return word_cleaned


import jieba

ka_list = []
comp_list = []


# 从包含中介词的文件中提取与给定种子词相关的关键词
def get_compkey_words(seedword: str):
    ka_list_line = []
    comp_list_line = []
    for midkeyword in midkeys_dict[seedword]:
        comkey_data = open('../data/temp/compkey_' + seedword, 'r', encoding='utf-8')
        # 将包含中介关键词的搜索数据进行分词
        compkey_query_list = []
        for line in comkey_data:
            if midkeyword['keyword'] in line:
                line = line.strip()
                word_seg = jieba.lcut(line)
                compkey_query_list.extend(word_seg)
        compkey_query_list = stop_words_filter(compkey_query_list)
        count_result = Counter(compkey_query_list)
        # 打印出现频率最高的三个词
        is_append = False
        for key, val in count_result.most_common(3):
            if key != midkeyword['keyword']:
                # 只把第一个词加入到ka_list_line和comp_list_line中
                if not is_append:
                    ka_list_line.append(val)
                    comp_list_line.append(key)
                    is_append = True
                # ka_list_line.append(val)
                # comp_list_line.append(key)
                print(key, val, end='  ')
        print()
        comkey_data.close()
    ka_list.append(ka_list_line)
    comp_list.append(comp_list_line)



In [48]:
for seedword in seedwords:
    print('\n' + seedword + ':\n')
    get_compkey_words(seedword)


图片:

做法 12907  名字 5374  
适合 3002  男生 2078  
2016 1183  款 1159  
句子 1495  女生 855  
价格表 8434  香烟 1775  
视频 3059  大全 1593  
年 32687  月 7079  
版 23197  下载 12019  
手机 10919  头像 10427  
头像 3193  日本 2290  

手机:

完整版 14343  破解版 13556  
txt 34432  全集 7514  
头像 10393  空间 5855  
6s 2825  钱 2492  
下载 3805  免费 1359  
荣耀 5292  p9 2574  
路由器 2824  电脑 1455  
高清 1021  图片 860  
教学 5205  观看 4749  
官网 1174  note 957  

小说:

火线 7135  王者 1631  
电影 2128  电视剧 1585  
电影 756  文 754  
2016 3775  前十名 2852  
耽 2050  美文 1935  
全文 22595  免费 13154  
txt 3433  末世 2227  
女主角 3792  男主角 1599  
txt 33049  手机 11863  
微微一笑 46657  电视剧 9269  

视频:

设计 832  教学计划 565  
观看 17397  阅读 4895  
在线 17301  电影 3568  
图片 31527  做法 12985  
版 22807  下载 11671  
游戏 1038  qq 708  
图片 3586  大全 1640  
教学 529  幼儿 241  
txt 34502  手机 11593  
荣耀 42487  英雄 3448  

下载:

重生 2059  微盘 1187  
版 18414  qq 10909  
电视剧 5397  漫画 3905  
手机 20907  完整版 14133  
手机 6621  手机软件 1866  
阅读 14470  全文 8616  
手机游戏 4107  大全 4030  
穿越 8198  好看 6087  
云 35215  资源 6506  


In [49]:
ka_list

[[12907, 3002, 1183, 1495, 8434, 3059, 32687, 23197, 10919, 3193],
 [14343, 34432, 10393, 2825, 3805, 5292, 2824, 1021, 5205, 1174],
 [7135, 2128, 756, 3775, 2050, 22595, 3433, 3792, 33049, 46657],
 [832, 17397, 17301, 31527, 22807, 1038, 3586, 529, 34502, 42487],
 [2059, 18414, 5397, 20907, 6621, 14470, 4107, 8198, 35215, 2125],
 [5514, 4265, 1958, 5128, 5659, 4677, 1268, 8809, 158703, 11275],
 [21907, 4737, 2172, 16373, 2050, 159379, 898, 3271, 2213, 34496],
 [876, 1011, 18237, 7889, 22745, 18837, 31519, 5240, 47855, 48585],
 [28837, 456, 32473, 1420, 2220, 2688, 350, 5892, 5982, 5201],
 [533, 2608, 16189, 5719, 21029, 23765, 86, 1745, 34060, 1000]]

In [50]:
comp_list

[['做法', '适合', '2016', '句子', '价格表', '视频', '年', '版', '手机', '头像'],
 ['完整版', 'txt', '头像', '6s', '下载', '荣耀', '路由器', '高清', '教学', '官网'],
 ['火线', '电影', '电影', '2016', '耽', '全文', 'txt', '女主角', 'txt', '微微一笑'],
 ['设计', '观看', '在线', '图片', '版', '游戏', '图片', '教学', 'txt', '荣耀'],
 ['重生', '版', '电视剧', '手机', '手机', '阅读', '手机游戏', '穿越', '云', '官方网站'],
 ['发型', '家常', '好听', '教学', '韩国', '下载', '下载', '倾城', '小说', '手机'],
 ['版', '女生', '重生', '查询', '侠盗', '小说', '163', '背景音乐', '女生', 'txt'],
 ['中国', '视频', '在线', '下载', '版', '观看', '图片', '小说', '倾城', '微微一笑'],
 ['最新', '新', '年', '没', '百度', '大学排名', '人物', '网', '约', '教学'],
 ['魔兽争霸', '月', 'wifi', '异界', '版', '手机', '学', '幸福生活', 'txt', '告白']]

In [51]:
# 竞争性关键词筛选
# 同时得到|{ka}|的值

ka_list = [[12907, 3002, 1183, 1495, 8434, 3059, 32687, 23197, 10919, 3193],
           [14343, 34432, 10393, 2825, 3805, 5292, 2824, 1021, 5205, 1174],
           [7135, 2128, 756, 3775, 2050, 22595, 3433, 3792, 33049, 46657],
           [832, 17397, 17301, 31527, 22807, 1038, 3586, 529, 34502, 42487],
           [2059, 18414, 5397, 20907, 6621, 14470, 4107, 8198, 35215, 2125],
           [5514, 4265, 1958, 5128, 5659, 4677, 1268, 8809, 158703, 11275],
           [21907, 4737, 2172, 16373, 2050, 159379, 898, 3271, 2213, 34496],
           [876, 1011, 18237, 7889, 22745, 18837, 31519, 5240, 47855, 48585],
           [28837, 456, 32473, 1420, 2220, 2688, 350, 5892, 5982, 5201],
           [533, 2608, 16189, 5719, 21029, 23765, 86, 1745, 34060, 1000]]

comp_list = [['做法', '适合', '2016', '句子', '价格表', '视频', '年', '版', '手机', '头像'],
             ['完整版', 'txt', '头像', '6s', '下载', '荣耀', '路由器', '高清', '教学', '官网'],
             ['火线', '电影', '电影', '2016', '耽', '全文', 'txt', '女主角', 'txt', '微微一笑'],
             ['设计', '观看', '在线', '图片', '版', '游戏', '图片', '教学', 'txt', '荣耀'],
             ['重生', '版', '电视剧', '手机', '手机', '阅读', '手机游戏', '穿越', '云', '官方网站'],
             ['发型', '家常', '好听', '教学', '韩国', '下载', '下载', '倾城', '小说', '手机'],
             ['版', '女生', '重生', '查询', '侠盗', '小说', '163', '背景音乐', '女生', 'txt'],
             ['中国', '视频', '在线', '下载', '版', '观看', '图片', '小说', '倾城', '微微一笑'],
             ['最新', '新', '年', '没', '百度', '大学排名', '人物', '网', '约', '教学'],
             ['魔兽争霸', '月', 'wifi', '异界', '版', '手机', '学', '幸福生活', 'txt', '告白']]

In [52]:
# 将种子关键词和竞争词合并成一个字典
compkeywords = dict(zip(seedwords, comp_list))
compkeywords

{'图片': ['做法', '适合', '2016', '句子', '价格表', '视频', '年', '版', '手机', '头像'],
 '手机': ['完整版', 'txt', '头像', '6s', '下载', '荣耀', '路由器', '高清', '教学', '官网'],
 '小说': ['火线', '电影', '电影', '2016', '耽', '全文', 'txt', '女主角', 'txt', '微微一笑'],
 '视频': ['设计', '观看', '在线', '图片', '版', '游戏', '图片', '教学', 'txt', '荣耀'],
 '下载': ['重生', '版', '电视剧', '手机', '手机', '阅读', '手机游戏', '穿越', '云', '官方网站'],
 '大全': ['发型', '家常', '好听', '教学', '韩国', '下载', '下载', '倾城', '小说', '手机'],
 'qq': ['版', '女生', '重生', '查询', '侠盗', '小说', '163', '背景音乐', '女生', 'txt'],
 '电影': ['中国', '视频', '在线', '下载', '版', '观看', '图片', '小说', '倾城', '微微一笑'],
 '中国': ['最新', '新', '年', '没', '百度', '大学排名', '人物', '网', '约', '教学'],
 '世界': ['魔兽争霸', '月', 'wifi', '异界', '版', '手机', '学', '幸福生活', 'txt', '告白']}

In [53]:
# 取出每个种子关键词的中介关键词的keyword，取成二维列表
midkeywords_list = []
for seedword in seedwords:
    midkeywords_list.append([midkey['keyword'] for midkey in midkeys_dict[seedword]])
midkeywords_list

[['大全', '发型', '报价', '唯美', '价格', '搞笑', '2016', '手机', 'qq', '动漫'],
 ['版', '下载', 'qq', '苹果', '软件', '华为', '设置', '壁纸', '视频', '小米'],
 ['穿越', '好看', '类似', '排行榜', '耽美', '阅读', '重生', '主角', '下载', '倾城'],
 ['教学', '在线', '观看', '大全', '手机', '腾讯', '搞笑', '舞蹈', '下载', '王者'],
 ['txt', '手机', '全集', '版', '软件', '免费', '游戏', '小说', '百度', '官方'],
 ['图片', '做法', '名字', '视频', '电影', '游戏', '歌曲', '电视剧', '说', 'qq'],
 ['手机', '头像', '空间', '号', '飞车', '说', '邮箱', '音乐', '网名', '下载'],
 ['韩国', '完整版', '观看', '全集', '手机', '在线', '大全', '好看', '微微一笑', '倾城'],
 ['新', '歌声', '2016', '声音', '地图', '排名', '关系', '官网', '奥运会', '视频'],
 ['魔兽', '两个', 'w', '异', '手机', '版', '从零开始', '生活', '下载', '完美']]

In [54]:
# 每个中介关键词对应的竞争性关键词
# 种子关键词：{竞争关键词：中介关键词}
compwords = dict.fromkeys(seedwords, {})
i = 0
for list in midkeywords_list:
    compwords[seedwords[i]] = dict(zip(list, comp_list[i]))
    i += 1
compwords

{'图片': {'大全': '做法',
  '发型': '适合',
  '报价': '2016',
  '唯美': '句子',
  '价格': '价格表',
  '搞笑': '视频',
  '2016': '年',
  '手机': '版',
  'qq': '手机',
  '动漫': '头像'},
 '手机': {'版': '完整版',
  '下载': 'txt',
  'qq': '头像',
  '苹果': '6s',
  '软件': '下载',
  '华为': '荣耀',
  '设置': '路由器',
  '壁纸': '高清',
  '视频': '教学',
  '小米': '官网'},
 '小说': {'穿越': '火线',
  '好看': '电影',
  '类似': '电影',
  '排行榜': '2016',
  '耽美': '耽',
  '阅读': '全文',
  '重生': 'txt',
  '主角': '女主角',
  '下载': 'txt',
  '倾城': '微微一笑'},
 '视频': {'教学': '设计',
  '在线': '观看',
  '观看': '在线',
  '大全': '图片',
  '手机': '版',
  '腾讯': '游戏',
  '搞笑': '图片',
  '舞蹈': '教学',
  '下载': 'txt',
  '王者': '荣耀'},
 '下载': {'txt': '重生',
  '手机': '版',
  '全集': '电视剧',
  '版': '手机',
  '软件': '手机',
  '免费': '阅读',
  '游戏': '手机游戏',
  '小说': '穿越',
  '百度': '云',
  '官方': '官方网站'},
 '大全': {'图片': '发型',
  '做法': '家常',
  '名字': '好听',
  '视频': '教学',
  '电影': '韩国',
  '游戏': '下载',
  '歌曲': '下载',
  '电视剧': '倾城',
  '说': '小说',
  'qq': '手机'},
 'qq': {'手机': '版',
  '头像': '女生',
  '空间': '重生',
  '号': '查询',
  '飞车': '侠盗',
  '说': '小说',
  '邮箱': '163',
 

In [55]:
# 将|{ka}|的值存在字典ka_query_list中
# 种子关键词：{竞争关键词：ka}
ka_query_volume = dict.fromkeys(seedwords, {})
i = 0
for list in midkeywords_list:
    ka_query_volume[seedwords[i]] = dict(zip(list, ka_list[i]))
    i += 1
ka_query_volume

{'图片': {'大全': 12907,
  '发型': 3002,
  '报价': 1183,
  '唯美': 1495,
  '价格': 8434,
  '搞笑': 3059,
  '2016': 32687,
  '手机': 23197,
  'qq': 10919,
  '动漫': 3193},
 '手机': {'版': 14343,
  '下载': 34432,
  'qq': 10393,
  '苹果': 2825,
  '软件': 3805,
  '华为': 5292,
  '设置': 2824,
  '壁纸': 1021,
  '视频': 5205,
  '小米': 1174},
 '小说': {'穿越': 7135,
  '好看': 2128,
  '类似': 756,
  '排行榜': 3775,
  '耽美': 2050,
  '阅读': 22595,
  '重生': 3433,
  '主角': 3792,
  '下载': 33049,
  '倾城': 46657},
 '视频': {'教学': 832,
  '在线': 17397,
  '观看': 17301,
  '大全': 31527,
  '手机': 22807,
  '腾讯': 1038,
  '搞笑': 3586,
  '舞蹈': 529,
  '下载': 34502,
  '王者': 42487},
 '下载': {'txt': 2059,
  '手机': 18414,
  '全集': 5397,
  '版': 20907,
  '软件': 6621,
  '免费': 14470,
  '游戏': 4107,
  '小说': 8198,
  '百度': 35215,
  '官方': 2125},
 '大全': {'图片': 5514,
  '做法': 4265,
  '名字': 1958,
  '视频': 5128,
  '电影': 5659,
  '游戏': 4677,
  '歌曲': 1268,
  '电视剧': 8809,
  '说': 158703,
  'qq': 11275},
 'qq': {'手机': 21907,
  '头像': 4737,
  '空间': 2172,
  '号': 16373,
  '飞车': 2050,
  '说': 159379,
  '邮

In [58]:
a_query_volume = dict.fromkeys(seedwords, {})
a_query_volume

{'图片': {},
 '手机': {},
 '小说': {},
 '视频': {},
 '下载': {},
 '大全': {},
 'qq': {},
 '电影': {},
 '中国': {},
 '世界': {}}

In [60]:
# 计算所有中介关键词的搜索量|{a}|
def count_midkeyword(seedword):
    # count_dict=dict.fromkeys(midkeywords[seedword],0)
    count_dict = dict.fromkeys(midkeywords_list[seedwords.index(seedword)], 0)
    query_data = open('../data/temp/cleaned.train', 'r', encoding='utf-8')
    for sentence in query_data:
        # for midkeyword in midkeywords[seedword]:
        for midkeyword in midkeywords_list[seedwords.index(seedword)]:
            if midkeyword in sentence:
                count_dict[midkeyword] += 1
    query_data.close()
    return count_dict


for seedword in seedwords:
    print(seedword)
    a_query_volume[seedword] = count_midkeyword(seedword)

图片
手机
小说
视频
下载
大全
qq
电影
中国
世界


In [61]:
a_query_volume

{'图片': {'大全': 136291,
  '发型': 23104,
  '报价': 16821,
  '唯美': 9691,
  '价格': 61652,
  '搞笑': 16286,
  '2016': 154015,
  '手机': 212970,
  'qq': 101505,
  '动漫': 34255},
 '手机': {'版': 164344,
  '下载': 136374,
  'qq': 101505,
  '苹果': 56148,
  '软件': 57257,
  '华为': 28290,
  '设置': 26360,
  '壁纸': 12547,
  '视频': 166709,
  '小米': 25082},
 '小说': {'穿越': 41807,
  '好看': 28755,
  '类似': 18051,
  '排行榜': 26951,
  '耽美': 17386,
  '阅读': 51664,
  '重生': 73955,
  '主角': 15254,
  '下载': 136374,
  '倾城': 60594},
 '视频': {'教学': 13989,
  '在线': 57125,
  '观看': 26462,
  '大全': 136291,
  '手机': 212970,
  '腾讯': 13723,
  '搞笑': 16286,
  '舞蹈': 8747,
  '下载': 136374,
  '王者': 51417},
 '下载': {'txt': 66076,
  '手机': 212970,
  '全集': 50141,
  '版': 164344,
  '软件': 57257,
  '免费': 51089,
  '游戏': 82161,
  '小说': 178270,
  '百度': 69246,
  '官方': 10041},
 '大全': {'图片': 300851,
  '做法': 59439,
  '名字': 35753,
  '视频': 166709,
  '电影': 110244,
  '游戏': 82161,
  '歌曲': 35632,
  '电视剧': 63199,
  '说': 292458,
  'qq': 101505},
 'qq': {'手机': 212970,
  '头像': 45738,
 

In [ ]:
sa_query_volume = dict.fromkeys(seedwords, {})


# 把midkey_dict中中的keyword和freq对应起来，存入sa_query_volume中
def get_sa_query_volume():
    for seedword in seedwords:
        sa_query_volume[seedword] = {}
        for midkeyword in midkeys_dict[seedword]:
            sa_query_volume[seedword][midkeyword['keyword']] = midkeyword['freq']
    return sa_query_volume


get_sa_query_volume()

sa_query_volume




In [ ]:

# 计算Comp
import copy


def getcomp():
    comp_query_volume = copy.deepcopy(sa_query_volume)
    for seedword in a_query_volume:
        for midkeyword in a_query_volume[seedword]:
            comp_query_volume[seedword][midkeyword] = ka_query_volume[seedword][midkeyword] / (
                    a_query_volume[seedword][midkeyword] - sa_query_volume[seedword][midkeyword])
    return comp_query_volume


result_query_volume = getcomp()
result_query_volume


In [ ]:
# 计算w_midkeyword
w_midkeyword = dict.fromkeys(seedwords, {})
for seedword in seedwords:
    w_midkeyword[seedword] = {}
    for midkeyword in midkeys_dict[seedword]:
        w_midkeyword[seedword][midkeyword['keyword']] = midkeyword['weight']
w_midkeyword

In [ ]:
def comp_result():
    result = {}
    for seedword in result_query_volume:
        result.setdefault(seedword, {})
        i = 0
        for midkeyword in result_query_volume[seedword]:
            result[seedword][compkeywords[seedword][i]] = w_midkeyword[seedword][midkeyword] * \
                                                          result_query_volume[seedword][midkeyword]
            i += 1
    return result


comp_k_s = comp_result()
#关键词k与种子关键词s的竞争性程度

# 每个种子关键词的竞争性关键词根据竞争度排序
for seedword in seedwords:
    comp_k_s[seedword] = dict(sorted(comp_k_s[seedword].items(), key=lambda x: x[1], reverse=True))
comp_k_s

In [ ]:
import pandas as pd


In [ ]:
# 根据竞争度排序后，把每个种子关键词的竞争性关键词和竞争度存储到字典中
comp_result_dict = {}
for seedword in seedwords:
    comp_result_dict[seedword] = {}
    # comp_result_dict[seedword][compkeywords[seedword]] = comp_k_s[seedword]
    for compword in compkeywords[seedword]:
        comp_result_dict[seedword][compword] = comp_k_s[seedword][compword]

comp_result_dict